# info_DCGANs example

## 초기 설정들

In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import time
import time 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

os.environ["CUDA_VISIBLE_DEVICES"]="1"


import pickle
import gzip

file_dir = 'anoGANs_MNIST_data/'

with gzip.open(file_dir + 'test_normal_data.pickle.gzip','rb') as f :
    test_normal_data = pickle.load(f)
    print('test_normal_data : ' ,test_normal_data.shape)

with gzip.open(file_dir + 'test_anomalous_data.pickle.gzip','rb') as f :
    test_anomalous_data = pickle.load(f)
    print('test_anomalous_data : ',test_anomalous_data.shape)
    
with gzip.open(file_dir + 'train_normal_data.pickle.gzip','rb') as f :
    train_normal_data = pickle.load(f)
    print('train_normal_data : ', train_normal_data.shape)
    
with gzip.open(file_dir + 'train_anomalous_data.pickle.gzip','rb') as f :
    train_anomalous_data = pickle.load(f)
    print('train_anomalous_data : ',train_anomalous_data.shape )

def idx_shuffle(x) : 
    l = x.shape[0]
    idx = np.arange(l)
    np.random.shuffle(idx)
    shuffled_x = np.empty(x.shape)

    for i in range(l):
        shuffled_x[idx[i]] = x[i]
    
    return shuffled_x

def mnist_4by4_save(samples,path):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)    
    gs.update(wspace=0.05, hspace=0.05) #이미지 사이간격 조절
  
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')    
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
   
        plt.imshow(sample.reshape(64, 64), cmap='Greys_r',clim=(0.0,1.0))
    plt.colorbar()
    plt.savefig(path, bbox_inches='tight')
    plt.close(fig)
   
    return None

def gan_loss_graph_save(G_loss,D_loss,path):
    x1 = range(len(G_loss))
    x2 = range(len(D_loss))
      
    y1 = G_loss
    y2 = D_loss
  
      
    plt.plot(x1,y1,label='G_loss') 
    plt.plot(x2,y2,label='D_loss') 
  
    plt.xlabel('weight per update')
    plt.ylabel('loss')             
    plt.legend(loc=4)              
    plt.grid(True)
    plt.tight_layout()
  
    plt.savefig(path)              

    return None

file_name = 'ex_info_DCGANs_v3'

if not os.path.isdir(file_name) :
    os.mkdir(file_name)


test_normal_data :  (5139, 64, 64, 1)
test_anomalous_data :  (4861, 64, 64, 1)
train_normal_data :  (28038, 64, 64, 1)
train_anomalous_data :  (26962, 64, 64, 1)


## 네트워크 정의

D부분을 encoder와 discriminator로 나눈 이유는 encoder를 나중에 feature map으로 쓰기 위해서 편의상 나누어서 정의함



In [2]:
train_epoch = 50
batch_size = 64
z_size = 100
c_size = 10


def G(x,c,isTrain = True, reuse = False, name = 'G') : #input = (minibatch * w * h * ch)
    
    # out size = (in size + 2*padding - kenel)/strides + 1    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)

     #x = (-1, 1, 1, 100)
    x_concat = tf.concat([x,c],3)
    with tf.variable_scope('G',reuse=reuse)  :
        
        #x = (-1, 1, 1, 100)

        conv1 = tf.layers.conv2d_transpose(x_concat, 1024, [4,4], strides=(1,1),padding = 'valid',
                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(tf.layers.batch_normalization(conv1,training=isTrain))#1024*4*4
        
        conv2 = tf.layers.conv2d_transpose(r1,512,[3,3], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#512*8*8
                
        conv3 = tf.layers.conv2d_transpose(r2,256,[3,3], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#256*16*16

        conv4 = tf.layers.conv2d_transpose(r3,128,[3,3], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init)
        r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain))#128*32*32

        conv5 = tf.layers.conv2d_transpose(r4,1,[3,3], strides=(2,2),padding = 'same',
                kernel_initializer=w_init, bias_initializer=b_init) #1*64*64
        
    r5= tf.nn.tanh(conv5,name=name)#1*64*64
  
    return r5

def D_enc(x,isTrain=True,reuse = False, name = 'D_enc') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('D_enc', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   

        conv1 = tf.layers.conv2d(x,128,[3,3], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init) 
        r1 = tf.nn.elu(conv1)#128*32*32

   
        conv2 = tf.layers.conv2d(r1,256,[3,3], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r2 = tf.nn.elu(tf.layers.batch_normalization(conv2,training=isTrain))#256*16*16

  
        conv3 = tf.layers.conv2d(r2,512,[3,3], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r3 = tf.nn.elu(tf.layers.batch_normalization(conv3,training=isTrain))#512*8*8
        
        conv4 = tf.layers.conv2d(r3,1024,[3,3], strides=(2,2),padding = 'same',
                                kernel_initializer=w_init, bias_initializer=b_init)
    r4 = tf.nn.elu(tf.layers.batch_normalization(conv4,training=isTrain), name = name)#1024*4*4
    return r4

def D_dis(x,isTrain=True,reuse = False, name = 'D_dis') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('D_dis', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   

        conv5 = tf.layers.conv2d(x,2048,[1,1], strides=(1,1),padding = 'valid',
                                kernel_initializer=w_init, bias_initializer=b_init)
        r5 = tf.nn.elu(tf.layers.batch_normalization(conv5,training=isTrain))#2048*1*1


        conv6 = tf.layers.conv2d(r5,1,[1,1], strides=(1,1),padding = 'valid',
                                kernel_initializer=w_init, bias_initializer=b_init)
      
    r6 = tf.nn.sigmoid(conv6,name=name)#1*1*1
    
    return r6

def Q_cat(x,reuse = False, name = 'Q_cat') :
    
    w_init = tf.truncated_normal_initializer(mean= 0.0, stddev=0.02)
    b_init = tf.constant_initializer(0.0)
    with tf.variable_scope('Q_cat', reuse=reuse) :
        
        #x = (-1,64,64,1)
        # out size = (in size + 2*padding - kenel)/strides + 1   
        x_flat = tf.reshape(x, (-1, 1024*4*4))
        
        w1_cat = tf.get_variable('w1_cat',[x_flat.get_shape()[1], c_size],initializer=w_init)
        b1_cat = tf.get_variable('b1_cat',[c_size],initializer=b_init)
                                          
    r_cat = tf.nn.softmax(tf.matmul(x_flat,w1_cat) + b1_cat, name = name)
    
    return tf.reshape(r_cat, (-1,1,1,c_size))

## 네트워크 연결

In [ ]:



z = tf.placeholder(tf.float32,shape=(None,1,1,z_size),name = 'z')    #x_z = G(z,c)
c_z = tf.placeholder(tf.float32,shape=(None,1,1,c_size),name = 'c_z')    #x_z = G(z,c)

u = tf.placeholder(tf.float32, shape = (None, 64,64,1),name='u')      #u = x
c_u = tf.placeholder(tf.float32,shape=(None,1,1,c_size),name = 'c_u')    #x_z = G(z,c)

isTrain = tf.placeholder(dtype=tf.bool,name='isTrain')  # BN 설정 parameter


G_sample = G(z,c_z,isTrain,name='G_sample') # G(z,c)



D_real = D_dis(D_enc(u, isTrain,reuse=False), isTrain, reuse=False, name = 'D_real')                       # D(x)
D_fake = D_dis(D_enc(G_sample, isTrain,reuse=True), isTrain, reuse=True, name='D_fake')         # D(G(z))

D_real_loss = tf.reduce_mean(-tf.log(D_real + 1e-8),name = 'D_real_loss')              # E[-log(D(x))] 
D_fake_loss = tf.reduce_mean(-tf.log(1 - D_fake + 1e-8),name = 'D_fake_loss')      # E[-log(1-D(G(z)))]


D_loss =  tf.add(D_real_loss,D_fake_loss,name='D_loss')                                        #  E[-log(D(x))]  + E[-log(1-D(G(z)))]

Q_fake = Q_cat(D_enc(G_sample, isTrain,reuse=True), reuse=False, name='Q_fake')
Q_real = Q_cat(D_enc(u, isTrain,reuse=True), reuse=True, name='Q_real')

Q_fake_loss = tf.reduce_mean(tf.reduce_sum(-c_z*tf.log(Q_fake + 1e-8), axis = (1,2,3)),name = 'Q_fake_loss')
Q_real_loss = tf.reduce_mean(tf.reduce_sum(-c_u*tf.log(Q_real + 1e-8), axis = (1,2,3)),name = 'Q_real_loss')

Q_loss = tf.add(Q_fake_loss, Q_real_loss, name ='Q_loss')

G_loss =  tf.reduce_mean(-tf.log(D_fake + 1e-8),name ='G_loss')                             

                                                                                                                                    

T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('D_dis') or var.name.startswith('D_enc')]
G_vars = [var for var in T_vars if var.name.startswith('G')]
Q_vars = [var for var in T_vars if var.name.startswith('Q')]
    
    # When using the batchnormalization layers,
    # it is necessary to manually add the update operations
    # because the moving averages are not included in the graph
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)) :        
    D_optim = tf.train.AdamOptimizer(1e-5,beta1=0.5).minimize(D_loss, var_list=D_vars, name='D_optim') 
    G_optim = tf.train.AdamOptimizer(1e-4,beta1=0.5).minimize(G_loss, var_list=G_vars, name='G_optim')
    Q_optim = tf.train.AdamOptimizer(1e-5,beta1=0.5).minimize(Q_loss, var_list=Q_vars, name='Q_optim')
    



## 학습

In [ ]:
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess :
    sess.run(tf.global_variables_initializer())

    np.random.seed(int(time.time()))
    one_hot = np.eye(c_size)
    temp2 = np.array([1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4])
    test_c = one_hot[temp2].reshape([-1,1,1,c_size])
    test_z = np.random.normal(0,0.1,size=(16,1,1,z_size))
    
    log_txt = open(file_name +'/log.txt','w')

    hist_G = []
    hist_D = []
    G_error = []
    D_error = []
    
    D_fake_error = []
    D_real_error = []
    
    Q_fake_error = []
    Q_real_error = []
    
    start = time.time()
    for epoch in range(train_epoch) :
        
        train_normal_data = idx_shuffle(train_normal_data) 
        one_hot = np.eye(c_size)
        
        for iteration in range(train_normal_data.shape[0] // batch_size) : 
        

            temp1 = np.random.randint(1,10,(batch_size))                                                                                                                                     
            c_z_ = one_hot[temp1].reshape([-1,1,1,c_size])
                   
            train_images = train_normal_data[iteration*batch_size : (iteration+1)*batch_size]      
            u_ = np.reshape(train_images,(-1,64,64,1)) 
            z_ = np.random.normal(0,1,size=(batch_size,1,1,z_size))
            
        
            _ , D_e,D_real_e,D_fake_e = sess.run([D_optim, D_loss,D_real_loss,D_fake_loss], {u : u_, z : z_,c_z : c_z_ ,isTrain : True})
            D_error.append(D_e)
            D_real_error.append(np.maximum(0.0, D_real_e))
            D_fake_error.append(np.maximum(0.0,D_fake_e))

            #    train_images,train_labels = mnist.train.next_batch(100)    
            #    u_ = np.reshape(train_images,(-1,64,64,1)) 
            #    z_ = np.random.normal(0,1,size=(100,1,1,100))
           
            temp2= np.random.randint(0,10,(batch_size))                                                                                                                                     
            c_z_ = one_hot[temp2].reshape([-1,1,1,c_size])
            _ , G_e = sess.run([G_optim, G_loss], {u : u_, z : z_, c_z : c_z_, isTrain : True}) 
            G_error.append(G_e)
        
                    
            temp3 = np.random.randint(0,1,(batch_size))                                                                                                                                     
            c_u_ = one_hot[temp3].reshape([-1,1,1,c_size])
            temp4 = np.random.randint(1,10,(batch_size))                                                                                                                                     
            c_z_ = one_hot[temp4].reshape([-1,1,1,c_size])
            
            _ , Q_real_e, Q_fake_e= sess.run([Q_optim, Q_real_loss, Q_fake_loss], {u : u_, z : z_, c_z : c_z_,c_u : c_u_ ,isTrain : True}) 
            Q_fake_error.append(Q_real_e)
            Q_real_error.append(Q_fake_e)
        
        hist_D.append(np.mean(D_error)) 
        hist_G.append(np.mean(G_error))
        r = sess.run([Q_real],feed_dict={u : test_normal_data[0:16], isTrain : False})     
        
        print(r)
        print('D_e : %.6f, D_real_e : %.6f, D_fake_e : %.6f, G_e : %.6f, Q_real_e : %6f, Q_fake_e : %.6f'%(np.mean(D_error), np.mean(D_real_error),
            np.mean(D_fake_error), np.mean(G_error), np.mean(Q_real_error), np.mean(Q_fake_error)))
        
        log_txt.write('D_e : %.6f, D_real_e : %.6f, D_fake_e : %.6f, G_e : %.6f\n'%(np.mean(D_error),
            np.mean(D_real_error), np.mean(D_fake_error), np.mean(G_error)))
      
        r = sess.run([G_sample],feed_dict={z : test_z, c_z : test_c, isTrain : False})        
        mnist_4by4_save(np.reshape(r,(-1,64,64,1)),file_name + '/result_{}.png'.format(str(epoch).zfill(3)))

        np.random.seed(int(time.time()))

        G_error = []
        D_error = []
        D_fake_error = []
        D_real_error = []
        Q_fake_error = []
        Q_real_error = []
 
    log_txt.close()
    gan_loss_graph_save(G_loss = hist_G,D_loss=hist_D,path = file_name + '/loss_graph.png')   
    saver = tf.train.Saver()
    saver.save(sess,file_name + '/para.cktp')
    
    end = time.time()-start

    print("total time : ",end)

[array([[[[ 0.3523154 ,  0.07942601,  0.08670408,  0.00954275,  0.125844  ,
           0.03558749,  0.11504138,  0.00985904,  0.15418673,  0.03149313]]],


       [[[ 0.49524188,  0.02816232,  0.02143966,  0.03944894,  0.05976787,
           0.11922996,  0.10609514,  0.04465223,  0.04231885,  0.04364309]]],


       [[[ 0.31398889,  0.22357382,  0.03615941,  0.01555804,  0.05920168,
           0.01883468,  0.24436346,  0.00606101,  0.07185216,  0.01040692]]],


       [[[ 0.93199569,  0.00252087,  0.0026896 ,  0.01756022,  0.00870932,
           0.01562276,  0.00693179,  0.00165518,  0.00850251,  0.00381207]]],


       [[[ 0.52239627,  0.0431437 ,  0.02502168,  0.04309506,  0.03560809,
           0.09093209,  0.07765497,  0.04675833,  0.04551208,  0.06987771]]],


       [[[ 0.76821661,  0.01359082,  0.03343935,  0.06112529,  0.08084341,
           0.00650781,  0.01495127,  0.00449573,  0.00891689,  0.00791289]]],


       [[[ 0.74428719,  0.00623141,  0.01999188,  0.09115684,  0.0708

## network load

In [ ]:
if not os.path.isdir(file_name) :
    os.mkdir(file_name)

    
sess = tf.InteractiveSession()
    
new_saver = tf.train.import_meta_graph(file_name + '/para.cktp.meta')
new_saver.restore(sess, tf.train.latest_checkpoint(file_name + '/'))


z = sess.graph.get_tensor_by_name("z:0")
u = sess.graph.get_tensor_by_name("u:0")

isTrain = sess.graph.get_tensor_by_name("isTrain:0")

G_sample = sess.graph.get_tensor_by_name("G_sample:0")


D_real_loss = sess.graph.get_tensor_by_name('D_real_loss:0')
D_fake_loss = sess.graph.get_tensor_by_name('D_fake_loss:0')

D_loss = sess.graph.get_tensor_by_name("D_loss:0")
G_loss = sess.graph.get_tensor_by_name("G_loss:0")


D_optim = sess.graph.get_operation_by_name("D_optim")
G_optim = sess.graph.get_operation_by_name("G_optim")





In [ ]:
  7.70249069e-01+7.43932128e-02+9.30449925e-03+2.12338828e-02+ 1.95606351e-02+ 1.22750308e-02+9.74359456e-03+  1.96765680e-02+   9.91886109e-03+5.36446124e-02

In [ ]:
np.array(r).shape

In [8]:
8*8*256

16384